#### EXAMPLE WRAPPING POLARS

In [1]:
import polars
from coppertop.pipe import *
# from coppertop.utils import *

from bones.lang.types import btup
from dm.core.types import date, N, num
import dm.pp
from broot import *
from broot import DD

ImportError: cannot import name 'btup' from 'bones.lang.types' (/Users/david/arwen/IR/minc/coppertop/src/bones/lang/types.py)

In [2]:
import builtins
from bones.ts.metatypes import weaken, BTAtom
from dm.core.types import txt, pylist, pytuple, pydict, T1
from dm.core.datetime import parseDate, toCTimeFormat


# pframe = BTAtom.ensure('pframe')
# pcol = BTAtom.ensure('pcol')

# weaken(polars.DataFrame, pframe)
# weaken(polars.Series, pcol)

pframe = polars.DataFrame
pcol = polars.Series


@coppertop(style=binary)
def take(a:pframe, k:txt) -> pframe:
    return a[[k]]

@coppertop(style=binary)
def take(a:pframe, isOrKs:pylist+pytuple) -> pframe:
    if not isOrKs:
        raise NotYetImplemented()
    elif isinstance(isOrKs[0], (builtins.str, txt)):
        return a[isOrKs]
    elif isinstance(isOrKs[0], int):
        raise NotYetImplemented()
    else:
        raise TypeError()

@coppertop(style=binary)
def to(xs:pcol, t:(N**num)[btup]) -> (N**num)[btup]:
    xs = xs.to_numpy()
    return btup(t, [parseNum(x) for x in xs])

@coppertop(style=binary)
def to(xs:pcol, t:(N**date)[btup], f:txt) -> (N**date)[btup]:
    xs = xs.to_numpy()
    cFormat = toCTimeFormat(f)
    return btup(t, [parseDate(x, cFormat) for x in xs])

@coppertop(style=ternary)
def atPut(pf:pframe, k:txt, col:pcol) -> pframe:
    # pf.replace is destructive :(, pf.with_column is non-destructive :)
    return pf.with_column(polars.Series(k, col))

@coppertop(style=ternary)
def atPut(pf:pframe, k:txt, col:(N**T1)[btup]) -> pframe:
    # pf.replace is destructive :(, pf.with_column is non-destructive :)
    return pf.with_column(polars.Series(k, col))

@coppertop
def csvTo(filename:txt, t:pframe) -> pframe:
    return polars.read_csv(filename)

@coppertop
def csvTo(filename:txt, t:pframe, renames:pydict) -> pframe:
    # rename is non-destuctive :)
    return polars.read_csv(filename).rename(renames)

@coppertop
def csvTo(filename:txt, t:pframe, renames:pydict, conversions:pydict) -> pframe:
    return polars.read_csv(filename)

@coppertop(style=unary)
def keys(d:pframe) -> pylist:
    return d.columns

In [3]:
renames = {
    'Date':'date', '1 Mo':'1m', '2 Mo':'2m', '3 Mo':'3m', '6 Mo':'6m', '1 Yr':'1y', '2 Yr':'2y', 
    '3 Yr':'3y', '5 Yr':'5y', '7 Yr':'7y', '10 Yr':'10y', '20 Yr':'20y', '30 Yr':'30y',
}

pdf = './analysis/data/US yields.csv' >> csvTo(_, pframe, renames)

In [4]:
pdf >> keys

['date',
 '1m',
 '2m',
 '3m',
 '6m',
 '1y',
 '2y',
 '3y',
 '5y',
 '7y',
 '10y',
 '20y',
 '30y']

In [5]:
pdf >> DD;

['__add__', '__bool__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__weakref__', '_comp', '_compare_to_non_df', '_compare_to_other_df', '_df', '_from_arrow', '_from_dict', '_from_dicts', '_from_numpy', '_from_pandas', '_from_pydf', '_from_records', '_pos_idx', '_pos_idxs', '_read_avro', '_read_csv', '_read_ipc', '_read_json', '_read_ndjson', '_read_parquet', '_repr_html_', 'apply', 'cleared', 'clone', 'columns', 'describe', 'drop', 'drop_in_place', 'drop_nulls', 'dtypes', 'estimated_size', 'explode', 'extend', 'fill_nan', 'fill_nu

In [6]:
pdf['date'] >> DD;

['__abs__', '__add__', '__and__', '__array__', '__array_ufunc__', '__bool__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__invert__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pow__', '__radd__', '__rand__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rmod__', '__rmul__', '__ror__', '__rpow__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__weakref__', '__xor__', '_arithmetic', '_comp', '_from_arrow', '_from_pandas', '_from_pyseries', '_pos_idxs', '_repeat', '_s', 'abs', 'alias', 'all', 'any', 'append', 'apply', 'arccos', 'arccosh', 'arcsin', 'arcsinh', 'arctan', 'arctanh', '

In [7]:
pdf >> take >> 'date'

date
str
"""01/02/18"""
"""01/03/18"""
"""01/04/18"""
"""01/05/18"""
"""01/08/18"""
"""01/09/18"""
"""01/10/18"""
"""01/11/18"""
"""01/12/18"""


In [8]:
pdf2 = pdf >> atPut >> 'date' >> (pdf['date'] >> to(_, _, 'MM/DD/YY') >> (N**date)[btup])

In [9]:
pdf2 >> take >> ['date', '1y', '5y']

date,1y,5y
object,f64,f64
2018-01-02,1.83,2.25
2018-01-03,1.81,2.25
2018-01-04,1.82,2.27
2018-01-05,1.8,2.29
2018-01-08,1.79,2.29
2018-01-09,1.78,2.33
2018-01-10,1.78,2.32
2018-01-11,1.77,2.32
2018-01-12,1.78,2.35


In [10]:
pdf >> take >> 'date'

date
str
"""01/02/18"""
"""01/03/18"""
"""01/04/18"""
"""01/05/18"""
"""01/08/18"""
"""01/09/18"""
"""01/10/18"""
"""01/11/18"""
"""01/12/18"""


<br>

I'm not very keen on the typing of this next piece of code as it's harder to follow what's going on:

In [11]:
pdf[['date', '1y']] >> TT;
pdf[['date']] >> TT;
pdf['date'] >> TT;

polars.internals.dataframe.frame.DataFrame
polars.internals.dataframe.frame.DataFrame
polars.internals.series.series.Series


As an example, consider what the type is of `pdf[x]`. You can't tell from the syntax of the snippet and first have to figure the type of `x`.